In [1]:
## Importing required libraries
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from time import time
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline
import warnings

warnings.simplefilter("ignore")

In [2]:
database = 'database.sqlite'
conn = sqlite3.connect(database)

In [3]:
#Fetching required data tables
country_data = pd.read_sql("SELECT * FROM Country;", conn)
league_data = pd.read_sql("SELECT * FROM League;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_attr_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
team_attr_data = pd.read_sql("SELECT * FROM Team_Attributes;", conn)

In [4]:
match_2015_season = match_data[match_data['season'] == '2015/2016']
epl_2015_season = match_2015_season[match_2015_season['country_id'] == 1729]

In [5]:
epl_2015_season

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
4388,4389,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987032,9825,8654,0,...,NaN,1.30,5.75,12.00,NaN,NaN,NaN,NaN,NaN,NaN
4389,4390,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987033,8678,10252,0,...,NaN,2.00,3.50,4.20,NaN,NaN,NaN,NaN,NaN,NaN
4390,4391,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987034,8455,10003,2,...,NaN,1.40,5.00,9.50,NaN,NaN,NaN,NaN,NaN,NaN
4391,4392,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987035,8668,9817,2,...,NaN,1.73,3.90,5.40,NaN,NaN,NaN,NaN,NaN,NaN
4392,4393,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987036,8197,8472,4,...,NaN,2.00,3.40,4.33,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4763,4764,1729,1729,2015/2016,9,2015-10-17 00:00:00,1988795,8466,8197,2,...,NaN,1.75,3.90,5.00,NaN,NaN,NaN,NaN,NaN,NaN
4764,4765,1729,1729,2015/2016,9,2015-10-19 00:00:00,1988796,10003,10194,0,...,NaN,2.05,3.40,4.10,NaN,NaN,NaN,NaN,NaN,NaN
4765,4766,1729,1729,2015/2016,9,2015-10-17 00:00:00,1988797,8586,8650,0,...,NaN,2.45,3.50,3.00,NaN,NaN,NaN,NaN,NaN,NaN
4766,4767,1729,1729,2015/2016,9,2015-10-17 00:00:00,1988798,9817,9825,0,...,NaN,6.25,4.20,1.60,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
season_sliced_columns = ['season', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal',\
                 'home_player_1', 'home_player_2','home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',\
                 'home_player_7', 'home_player_8', 'home_player_9','home_player_10', 'home_player_11', 'away_player_1',\
                 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5','away_player_6', 'away_player_7',\
                 'away_player_8', 'away_player_9','away_player_10', 'away_player_11']

In [7]:
epl_2015_season_sliced = epl_2015_season[season_sliced_columns]

In [8]:
epl_2015_season_sliced

,season,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
4388,2015/2016,1987032,9825,8654,0,2,30859.0,26154.0,35606.0,46539.0,...,109897.0,35110.0,49543.0,155782.0,37169.0,575789.0,148302.0,25496.0,18506.0,192899.0
4389,2015/2016,1987033,8678,10252,0,1,30974.0,35515.0,24625.0,156013.0,...,183500.0,24208.0,161414.0,473853.0,261313.0,179410.0,182223.0,23991.0,154280.0,23264.0
4390,2015/2016,1987034,8455,10003,2,2,170323.0,31306.0,23783.0,30627.0,...,111800.0,155050.0,24948.0,102356.0,127130.0,144996.0,95955.0,157729.0,52563.0,26344.0
4391,2015/2016,1987035,8668,9817,2,2,31465.0,77690.0,263653.0,23268.0,...,213809.0,41927.0,40548.0,35712.0,41365.0,30966.0,24915.0,37411.0,71724.0,72436.0
4392,2015/2016,1987036,8197,8472,4,2,37770.0,67850.0,38899.0,23571.0,...,22964.0,26108.0,165526.0,180330.0,35443.0,25150.0,109058.0,24159.0,30348.0,42598.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4763,2015/2016,1988795,8466,8197,2,2,35496.0,160627.0,26552.0,209405.0,...,49571.0,23571.0,38899.0,43061.0,139671.0,173317.0,319300.0,214570.0,20694.0,286119.0
4764,2015/2016,1988796,10003,10194,0,1,30973.0,89185.0,155050.0,24948.0,...,34036.0,119541.0,200962.0,37194.0,23253.0,39109.0,176300.0,96643.0,110148.0,172321.0
4765,2015/2016,1988797,8586,8650,0,0,26295.0,159833.0,160599.0,37762.0,...,157838.0,22764.0,94043.0,314605.0,95327.0,307021.0,38807.0,184536.0,37234.0,426202.0
4766,2015/2016,1988798,9817,9825,0,3,30455.0,213809.0,41927.0,40548.0,...,427438.0,35606.0,46539.0,38521.0,159594.0,37436.0,75489.0,36378.0,50047.0,31013.0


### We want to handle 2015/16 season

In [9]:
starting_date = '2015-09-01'
ending_date = '2016-01-01'

In [10]:
higher_up = player_attr_data[player_attr_data['date'] > starting_date]
player_overall_df = higher_up[higher_up['date'] <  ending_date].drop_duplicates(subset = ["player_api_id"])


## currently we are only concerned with overall_rating

player_overall_df = player_overall_df[['player_api_id', 'overall_rating']]

In [11]:
player_overall_df

,player_api_id,overall_rating
1,505942,67.0
8,155782,73.0
39,162549,66.0
65,30572,69.0
87,23780,70.0
...,...,...
183823,107281,73.0
183856,491794,58.0
183878,99031,80.0
183896,192132,64.0


## Join Match Data and Player Overall

In [27]:
experimenting_df = epl_2015_season_sliced

In [28]:
experimenting_df

,season,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
4388,2015/2016,1987032,9825,8654,0,2,30859.0,26154.0,35606.0,46539.0,...,109897.0,35110.0,49543.0,155782.0,37169.0,575789.0,148302.0,25496.0,18506.0,192899.0
4389,2015/2016,1987033,8678,10252,0,1,30974.0,35515.0,24625.0,156013.0,...,183500.0,24208.0,161414.0,473853.0,261313.0,179410.0,182223.0,23991.0,154280.0,23264.0
4390,2015/2016,1987034,8455,10003,2,2,170323.0,31306.0,23783.0,30627.0,...,111800.0,155050.0,24948.0,102356.0,127130.0,144996.0,95955.0,157729.0,52563.0,26344.0
4391,2015/2016,1987035,8668,9817,2,2,31465.0,77690.0,263653.0,23268.0,...,213809.0,41927.0,40548.0,35712.0,41365.0,30966.0,24915.0,37411.0,71724.0,72436.0
4392,2015/2016,1987036,8197,8472,4,2,37770.0,67850.0,38899.0,23571.0,...,22964.0,26108.0,165526.0,180330.0,35443.0,25150.0,109058.0,24159.0,30348.0,42598.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4763,2015/2016,1988795,8466,8197,2,2,35496.0,160627.0,26552.0,209405.0,...,49571.0,23571.0,38899.0,43061.0,139671.0,173317.0,319300.0,214570.0,20694.0,286119.0
4764,2015/2016,1988796,10003,10194,0,1,30973.0,89185.0,155050.0,24948.0,...,34036.0,119541.0,200962.0,37194.0,23253.0,39109.0,176300.0,96643.0,110148.0,172321.0
4765,2015/2016,1988797,8586,8650,0,0,26295.0,159833.0,160599.0,37762.0,...,157838.0,22764.0,94043.0,314605.0,95327.0,307021.0,38807.0,184536.0,37234.0,426202.0
4766,2015/2016,1988798,9817,9825,0,3,30455.0,213809.0,41927.0,40548.0,...,427438.0,35606.0,46539.0,38521.0,159594.0,37436.0,75489.0,36378.0,50047.0,31013.0


In [29]:
columns_to_loop = ['home_player_1', 'home_player_2','home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',\
                 'home_player_7', 'home_player_8', 'home_player_9','home_player_10', 'home_player_11', 'away_player_1',\
                 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5','away_player_6', 'away_player_7',\
                 'away_player_8', 'away_player_9','away_player_10', 'away_player_11']

In [30]:
column_names_overall = ['home_player_1_overall', 'home_player_2_overall','home_player_3_overall', 'home_player_4_overall', 'home_player_5_overall', 'home_player_6_overall',\
                 'home_player_7_overall', 'home_player_8_overall', 'home_player_9_overall','home_player_10_overall', 'home_player_11_overall', 'away_player_1_overall',\
                 'away_player_2_overall', 'away_player_3_overall', 'away_player_4_overall', 'away_player_5_overall','away_player_6_overall', 'away_player_7_overall',\
                 'away_player_8_overall', 'away_player_9_overall','away_player_10_overall', 'away_player_11_overall']

In [31]:
experimenting_df.columns

Index(['season', 'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_1', 'home_player_2',
       'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',
       'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10',
       'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3',
       'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7',
       'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11'],
      dtype='object')

In [32]:
for i in range(len(column_names_overall)):
    experimenting_df = experimenting_df.merge(player_overall_df,\
                                left_on= columns_to_loop[i], right_on='player_api_id', how='left', suffixes=('_1', '_2'))\
                                .rename(columns={"overall_rating": column_names_overall[i]})

In [33]:
experimenting_df.columns

Index(['season', 'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_1', 'home_player_2',
       'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',
       'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10',
       'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3',
       'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7',
       'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11',
       'player_api_id_1', 'home_player_1_overall', 'player_api_id_2',
       'home_player_2_overall', 'player_api_id_1', 'home_player_3_overall',
       'player_api_id_2', 'home_player_4_overall', 'player_api_id_1',
       'home_player_5_overall', 'player_api_id_2', 'home_player_6_overall',
       'player_api_id_1', 'home_player_7_overall', 'player_api_id_2',
       'home_player_8_overall', 'player_api_id_1', 'home_player_9_overall',
       'player_api_id_2', 'home_p

### Create WIN, LOSE, DRAW labels

In [34]:
conditions = [
    (experimenting_df['home_team_goal'] > experimenting_df['away_team_goal']),
    (experimenting_df['home_team_goal'] == experimenting_df['away_team_goal']),
    (experimenting_df['home_team_goal'] < experimenting_df['away_team_goal']),
    ]

conditions_values = ["win", "draw", "lose"]

experimenting_df['results'] = np.select(conditions, conditions_values)


### Obtain Final Prepared Dataset

In [35]:
final_df_columns = ['home_player_1_overall', 'home_player_2_overall','home_player_3_overall', 'home_player_4_overall', 'home_player_5_overall', 'home_player_6_overall',\
                 'home_player_7_overall', 'home_player_8_overall', 'home_player_9_overall','home_player_10_overall', 'home_player_11_overall', 'away_player_1_overall',\
                 'away_player_2_overall', 'away_player_3_overall', 'away_player_4_overall', 'away_player_5_overall','away_player_6_overall', 'away_player_7_overall',\
                 'away_player_8_overall', 'away_player_9_overall','away_player_10_overall', 'away_player_11_overall', 'results']

In [36]:
prepared_df = experimenting_df[final_df_columns]

In [37]:
prepared_df

,home_player_1_overall,home_player_2_overall,home_player_3_overall,home_player_4_overall,home_player_5_overall,home_player_6_overall,home_player_7_overall,home_player_8_overall,home_player_9_overall,home_player_10_overall,...,away_player_3_overall,away_player_4_overall,away_player_5_overall,away_player_6_overall,away_player_7_overall,away_player_8_overall,away_player_9_overall,away_player_10_overall,away_player_11_overall,results
0,85.0,79.0,83.0,83.0,80.0,79.0,85.0,82.0,87.0,79.0,...,79.0,79.0,73.0,76.0,62.0,75.0,81.0,76.0,75.0,lose
1,76.0,72.0,71.0,72.0,71.0,74.0,71.0,73.0,71.0,69.0,...,79.0,76.0,75.0,76.0,75.0,74.0,76.0,76.0,74.0,lose
2,86.0,80.0,83.0,85.0,82.0,84.0,87.0,82.0,84.0,89.0,...,78.0,81.0,74.0,77.0,77.0,79.0,78.0,77.0,78.0,draw
3,80.0,81.0,77.0,81.0,66.0,79.0,80.0,80.0,77.0,80.0,...,76.0,71.0,77.0,77.0,76.0,66.0,75.0,77.0,75.0,draw
4,76.0,72.0,78.0,73.0,71.0,70.0,72.0,68.0,75.0,78.0,...,78.0,73.0,74.0,74.0,77.0,75.0,78.0,79.0,79.0,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,71.0,78.0,81.0,77.0,78.0,77.0,76.0,78.0,75.0,79.0,...,73.0,78.0,74.0,71.0,72.0,76.0,68.0,78.0,75.0,draw
376,78.0,76.0,78.0,81.0,74.0,77.0,77.0,64.0,79.0,77.0,...,76.0,76.0,74.0,76.0,76.0,80.0,80.0,79.0,75.0,lose
377,85.0,79.0,81.0,82.0,77.0,72.0,80.0,80.0,83.0,78.0,...,81.0,81.0,77.0,79.0,75.0,80.0,84.0,80.0,76.0,draw
378,75.0,77.0,76.0,71.0,69.0,77.0,71.0,73.0,75.0,66.0,...,83.0,83.0,80.0,79.0,85.0,82.0,87.0,86.0,81.0,lose


In [40]:
prepared_df.shape

(380, 23)